# Trotter Simulation 
Here we perfrom Quantum simulation of quantum system using the Trotter-Suzuki formula to decompose the exponential of a sum
of operators. The concepts and methods used in this notebook can be found in "Simulating Hamiltonian Dynamics Using Product Formulas", Omar Al Saeed. 

The most straightforward way of applying the time evolution operator $U(t)$ on some arbitrary initial state $|\phi(0)\rangle $ is to diagonalize the operator $H$ However, the basis of the diagonalized representation is often nontrivial, especially in cases where the Hamiltonian is a sum of non-commuting terms. However, using the Trotter-Suzuki formula we can approximate the multiplication of two non-commmuting operaters in the exponent to:

$$ e^{xAB} = \lim_{n \to 0}(e^{xA/n}e^{xB/n})^n $$
Where A and B are operators. 

The hamiltonian studied in this notbook is:
\begin{equation}H = -\sum_{i,j}J_{i,j}\sigma_i^z \sigma_j^z - h_j \sum\sigma_j^x  \end{equation}

The $\sigma_z$ terms do commute with each other, but don't commute with the $ \sigma_x $ terms. For the purpose of this paper, we will take $ A = \sum_{i,j}J_{i,j}\sigma_i^z \sigma_j^z$ and $B = h_j \sum\sigma_j^x $

In [39]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import *
from scipy.linalg import expm
from numpy.linalg import matrix_power

from scipy.integrate import ode

# for analytical exponentiation
from scipy.linalg import expm 
print("done")

done


In [51]:
def trotter_sim(J, h, initial_state, time, trotter_num):
    
    initial_state = np.asarray(initial_state)
    
    N = len(h)
    sx = np.array([[0, 1], [1, 0]])
    sz = np.array([[1, 0], [0, -1]])
    I = np.array([[1, 0], [0, 1]])
    
    sum_sx = []
    # build h times sum sigma_j^x
    for i in range(N):
        # make a list of N qeye matrices for one qubit
        op_list = []
        for _ in range(N):
            op_list.append(qeye(2))  
        # replace the jth element with sigmax, tensor, then append sum_sx
        op_list[i] = sigmax()
        sum_sx.append(h[i]*tensor(op_list))
        
    B = 0
    for op in sum_sx:
        B -= op
    B = np.asarray(B)
#     B = np.matmul(h, B)
    
    sum_sz_sz = []
    # bild sum J_{i,j} sigma_i^z sigma_j^z, we ignore the case where i = j
    for i in range(N):      
        for j in range(N):
            if i == j:
                continue
            # make a list of N qeye matrices for one qubit
            op_list = []
            for _ in range(N):
                op_list.append(qeye(2))  
            # replace the ith and jth elements with sigma_z, tensor, then append to sum_sz_sz
            op_list[i] = sigmaz()
            op_list[j] = sigmaz()
            sum_sz_sz.append(J[i][j]*tensor(op_list))
            
    A = 0        
    for op in sum_sz_sz:
        A -= op
    A = np.asarray(A)
    
    U = np.matmul(np.exp(-1j*A*time/trotter_num), np.exp(-1j*B*time/trotter_num))
    U = matrix_power(U, trotter_num)
    
    return np.matmul(U, initial_state)
    
    
    
    
h = np.array([1, 1])
J = np.array([[1, 1], [1, 1]])
initial_state = tensor(basis(2, 0), basis(2, 0))

trotter_sim(J, h, initial_state, 5, 100)
    
    
        

    

[[-2.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  2.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -2.+0.j]]


array([[-4.51417953e+119+3.18918183e+119j],
       [-4.34925538e+119+3.41067779e+119j],
       [-4.34925538e+119+3.41067779e+119j],
       [-4.51417953e+119+3.18918183e+119j]])

In [22]:
sigma_z = np.array([[1, 0], [0, -1]])
h = sigma_z
print(h)
print(h[0])
print(h[1])
print(h[0]*h[1])
# print(np.matmul(np.array([[1], [0]]), h[1]))
h[0].T

[[ 1  0]
 [ 0 -1]]
[1 0]
[ 0 -1]
[0 0]


array([1, 0])